In [ ]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import urllib
from datetime import date

config, context, api_url = postgres_to_girder.config()

girder_connection=postgres_to_girder.connect_to_girder(
    api_url=api_url,
    authentication=(
        config["girder"]["user"],
        config["girder"]["password"]
    )
)
conn = postgres_to_girder.connect_to_postgres(
    config["postgres"]
)

activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="Activities",
    girder_connection=girder_connection
)

postgres_tables = {
    table: pd.io.sql.read_sql_query(
        "SELECT * FROM {0};".format(
            table
        ),
        conn
    ) for table in {
        "acts",
        "users",
        "user_acts",
        "organizations",
        "answers"
    }
}

users_emails = postgres_to_girder.postgres_users_to_girder_users(
    postgres_tables["users"],
    girder_connection,
    config["missing_persons"]
)

postgres_tables["acts"] = postgres_to_girder._respondents(
    postgres_tables["acts"]
)

Connected to the Girder database 🏗🍃 and authenticated.
Connected to the Postgres database 🐘


In [14]:
acts = postgres_tables["acts"]
gc = girder_connection
users = postgres_tables["users"]

In [17]:
activities = postgres_to_girder.postgres_activities_to_girder_activities(
    acts,
    gc,
    users,
    users_emails,
    context
)

In [23]:
print({k for d in acts["act_data"] for k in json.loads(d)})

{'questions', 'accordion', 'answers', 'mode', 'frequency', 'instruction', 'audio_path', 'image_url', 'audio_url', 'timer'}


In [ ]:
for timing in {
    "1d": "daily",
    "1": "once",
    "8h": "3×daily",
    "12h": "2×daily"
}:
    print(timing)

In [ ]:
a

In [ ]:
b